change rsi_avg color 
- https://code.luasoftware.com/tutorials/algo-trading/mplfinance-change-color-depending-on-value-is-positive-or-negative/

---

# RSI study of  winning stocks 

---


In [9]:
# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [10]:
%matplotlib inline
import pandas as pd
import numpy as np

In [11]:
import mplfinance as mpf
mpf.__version__
# mpf.plot(df,volume=True,type='candle')


'0.12.8b10'

In [12]:

RSI_PERIOD, RSI_AVG, RSI_BAND_WIDTH = 100, 25, 0.6
EMA_FAST, EMA_SLOW, EMA_LONG = 15, 50, 150
EMA_FAST_SCALE = 1.1  # EMA10 band half-width factor
EMA_SLOW_SCALE = 2.0 
MA_VOL = 20
PANID_PRICE, PANID_VOL, PANID_RSI = 0, 1, 2
FLAG_DEBUG = False # True # 

In [13]:
def _ta_RSI(df, n=14):
    # https://github.com/wgong/mplfinance/blob/master/examples/rsi.py
    diff = df.w_p.diff().values
    gains = diff
    losses = -diff
    with np.errstate(invalid='ignore'):
        gains[(gains<0)|np.isnan(gains)] = 0.0
        losses[(losses<=0)|np.isnan(losses)] = 1e-10 # we don't want divide by zero/NaN
    m = (n-1) / n
    ni = 1 / n
    g = gains[n] = np.nanmean(gains[:n])
    l = losses[n] = np.nanmean(losses[:n])
    gains[:n] = losses[:n] = np.nan
    for i,v in enumerate(gains[n:],n):
        g = gains[i] = ni*v + m*g
    for i,v in enumerate(losses[n:],n):
        l = losses[i] = ni*v + m*l
    rs = gains / losses
    return 100 - (100/(1+rs)) - 50

def load_stooq_data(ticker):
    df = pd.read_csv(f'data/{ticker}.us.txt',index_col=2,parse_dates=True)
    df.reset_index(inplace=True)
    df.drop(columns=['<TICKER>', '<PER>', '<TIME>','<OPENINT>'], axis=1, inplace=True)
    df.rename(columns={'<DATE>' : "Date", '<OPEN>': "Open", '<HIGH>':"High", 
                   '<LOW>':"Low", '<CLOSE>':"Close", '<VOL>':"Volume"} ,inplace=True)
    df.set_index("Date", inplace=True)
    return df



def calculate_ta(df,rsi_period=RSI_PERIOD, avg_period=RSI_AVG, band_width=RSI_BAND_WIDTH):
    
    df["w_p"] = 0.25*(2*df["Close"] + df["High"] + df["Low"])
    df["ema_fast"] = df.w_p.ewm(span=EMA_FAST).mean()
    df["ema_slow"] = df.w_p.ewm(span=EMA_SLOW).mean()
    df["ema_long"] = df.w_p.ewm(span=EMA_LONG).mean()

    # range
    hl_mean_fast = (df.High - df.Low).ewm(span=int(EMA_FAST/2)).mean()
    df["ema_fast_u"] =  df.ema_fast + 0.5*hl_mean_fast * EMA_FAST_SCALE
    df["ema_fast_d"] =  df.ema_fast - 0.5*hl_mean_fast * EMA_FAST_SCALE

    hl_mean_slow = (df.High - df.Low).ewm(span=int(EMA_SLOW/2)).mean()
    df["ema_slow_u"] =  df.ema_slow + 0.5*hl_mean_slow * EMA_SLOW_SCALE
    df["ema_slow_d"] =  df.ema_slow - 0.5*hl_mean_slow * EMA_SLOW_SCALE

    # trim volume to avoid exponential form
    df['Volume'] = df['Volume'] / 1000000
    df["vol_avg"] = df.Volume.ewm(span=MA_VOL).mean()

    df["rsi"] = _ta_RSI(df,n=rsi_period)

    df["rsi_avg"] = df.rsi.ewm(span=avg_period).mean()
    df["rsi_u"] = df["rsi_avg"] + band_width 
    df["rsi_d"] = df['rsi_avg'] - band_width 

    return df

def _title_xy(ticker):
    # position title manually
    return {"title": f"{ticker.upper()}",  "x": 0.75, "y": 0.95}

def gen_chart(df_1, ticker, num, panid_price=PANID_PRICE, panid_vol=PANID_VOL, panid_rsi=PANID_RSI):
    light_black = '#8F8E83'

    ema_fast_u_plot = mpf.make_addplot(df_1["ema_fast_u"], panel=panid_price, color=light_black, linestyle="solid")
    ema_fast_d_plot = mpf.make_addplot(df_1["ema_fast_d"], panel=panid_price, color=light_black, linestyle="solid")
    ema_slow_plot = mpf.make_addplot(df_1["ema_slow"], panel=panid_price, color='b', width=2, linestyle="solid")

    ema_long_plot = mpf.make_addplot(df_1["ema_long"], panel=panid_price, width=2, color='k')  # magenta
    
    rsi_min = df_1.min(axis=0)[["rsi"]].min()
    rsi_max = df_1.max(axis=0)[["rsi"]].max()

    # change rsi_avg color 
    rsi_colors = ['#DCF7E5' if v >= 0 else '#F7D0CA'  for v in df_1["rsi_avg"]]
    rsi_avg_plot = mpf.make_addplot(df_1["rsi_avg"], panel=panid_rsi, type="bar", color=rsi_colors, 
                                    ylim=(rsi_min,rsi_max))
    rsi_avg_plot2 = mpf.make_addplot(df_1["rsi_avg"], panel=panid_rsi, color="b", width=1, linestyle="solid",
                                    ylim=(rsi_min,rsi_max))
    
    rsi_plot = mpf.make_addplot(df_1["rsi"], panel=panid_rsi, color='r', width=1, #title=f"{ticker}-RSI",
                                fill_between=dict(y1=df_1["rsi_d"].values,y2=df_1["rsi_u"].values,alpha=0.15,color='b'),
                                ylim=(rsi_min,rsi_max))
    # volume
    vol_avg_plot = mpf.make_addplot(df_1["vol_avg"], panel=panid_vol, color='k')

    plots = [
            # panel-0
            ema_fast_u_plot, ema_fast_d_plot, 
            ema_slow_plot, ema_long_plot , # ema_slow_u_plot, ema_slow_d_plot, 
            # panel-1
            # rsi_50_plot, 
            rsi_avg_plot, rsi_plot, rsi_avg_plot2, # , rsi_u_plot, rsi_d_plot 
            vol_avg_plot
        ]
    file_img = f"images/{ticker}-{str(num).zfill(2)}.png"
    print(file_img)
    mpf.plot(df_1, type='candle', 
            style='yahoo', 
            fill_between=dict(y1=df_1["ema_fast_d"].values,y2=df_1["ema_fast_u"].values,alpha=0.15,color='b'),
            panel_ratios=(6,1,6),
            addplot=plots, 
            title=_title_xy(ticker),
            volume=True, volume_panel=panid_vol, 
            ylabel="", ylabel_lower='',
            xrotation=0,
            datetime_format='%Y-%m-%d',
            savefig=file_img,
            figsize=(FIGURE_WIDTH,FIGURE_HEIGHT),
            tight_layout=True,
            show_nontrading=True
        )  

In [14]:
for ticker in ["intc", "googl", "aapl", "adbe", "amd", "amzn"]:

    # test
    if FLAG_DEBUG and ticker != "googl" : continue

    df = load_stooq_data(ticker)
    df = calculate_ta(df)
    
    LEN = df.shape[0]
    SPAN, OVERLAP = 200, 40
    FIGURE_WIDTH, FIGURE_HEIGHT =  16, 14

    num = 0
    intervals = []
    for ix_stop in range(LEN, RSI_PERIOD, -SPAN):
        ix_start = ix_stop - SPAN -int(OVERLAP/2)
        intervals.append((ix_start if ix_start > 0 else 0, ix_stop+int(OVERLAP/2)))    

    for ix_start,ix_stop in intervals[::-1]:
        num += 1

        # test
        if FLAG_DEBUG and num > 10:  break
        
        df_1 = df.iloc[ix_start : ix_stop]
        gen_chart(df_1, ticker, num)

images/intc-01.png
images/intc-02.png
images/intc-03.png
images/intc-04.png
images/intc-05.png
images/intc-06.png
images/intc-07.png
images/intc-08.png
images/intc-09.png
images/intc-10.png
images/intc-11.png
images/intc-12.png
images/intc-13.png
images/intc-14.png
images/intc-15.png
images/intc-16.png
images/intc-17.png
images/intc-18.png
images/intc-19.png
images/intc-20.png
images/intc-21.png
images/intc-22.png
images/intc-23.png
images/intc-24.png
images/intc-25.png
images/intc-26.png
images/intc-27.png
images/intc-28.png
images/intc-29.png
images/intc-30.png
images/intc-31.png
images/intc-32.png
images/intc-33.png
images/intc-34.png
images/intc-35.png
images/intc-36.png
images/intc-37.png
images/intc-38.png
images/intc-39.png
images/intc-40.png
images/intc-41.png
images/intc-42.png
images/intc-43.png
images/intc-44.png
images/intc-45.png
images/intc-46.png
images/intc-47.png
images/intc-48.png
images/intc-49.png
images/intc-50.png
images/intc-51.png
images/intc-52.png
images/intc-

## resample